In [1]:
import numpy as np
import pandas as pd 
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import pairwise_distances

# Import Data

In [3]:
# import data
ratings = pd.read_csv('rtg.csv')

In [1]:
# import kolom ke variabel
weight = ratings[['userId', 'movieId', 'S']]

NameError: name 'ratings' is not defined

In [27]:
# plot isi data ke matrix
llor = weight.pivot(index='userId', columns='movieId', values='S').fillna(0)
llor_matrix = csr_matrix(llor.values)

In [28]:
llor.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,3.300000,0.0,4.1,0.0,0.0,6.1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2.806738,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
llor_array = llor.values

In [30]:
user_similarity = pairwise_distances(llor_array, metric='cosine')

In [31]:
pd.DataFrame(user_similarity).head()

,0,1,2,3,4,5,6,7,8,9,...,600,601,602,603,604,605,606,607,608,609
0,0.000000,0.964411,0.956095,0.810081,0.883501,0.879927,0.852225,0.903215,0.940318,0.978408,...,0.950336,0.858962,0.785803,0.934751,0.856711,0.839131,0.764002,0.733569,0.911836,0.864302
1,0.964411,0.000000,1.000000,0.996529,0.979274,0.976359,0.965777,0.966424,1.000000,0.931499,...,0.834188,0.974649,0.984538,1.000000,1.000000,0.970164,0.990256,0.955526,0.970249,0.890105
2,0.956095,1.000000,0.000000,0.997158,0.994729,0.993758,1.000000,0.993169,1.000000,1.000000,...,0.996614,0.995137,0.969776,1.000000,0.995440,0.987582,0.968203,0.978519,1.000000,0.966506
3,0.810081,0.996529,0.997158,0.000000,0.852985,0.913670,0.890809,0.946888,0.993457,0.975955,...,0.924201,0.876086,0.706898,0.947117,0.916295,0.816079,0.882989,0.866015,0.960371,0.895524
4,0.883501,0.979274,0.994729,0.852985,0.000000,0.702546,0.889687,0.556787,1.000000,0.963089,...,0.938875,0.578644,0.877510,0.765854,0.882050,0.903695,0.864426,0.856693,0.705179,0.939093


In [90]:
# menghitung user 0
K = 60
pred_ratings = np.zeros(K)

# mengambil similarity paling kecil dari index ke 0 (user ke 0)
selected_similar = user_similarity[0][:]
# lakukan sorting mulai dari yang paling terkecil hingga paling terbesar (ascending)
sorted_similar, index_sorted = np.sort(selected_similar), np.argsort(selected_similar)
# ambil nilai sebanyak K untuk mendapatkan similarity yang paling terdekat
most_similar, index_similar = sorted_similar[:K+1], index_sorted[:K+1]
# hapus nilai diindex paling awal karena pada index tersebut user menghitung dirinya sendiri
most_similar, index_similar = np.delete(most_similar, 0), np.delete(index_similar, 0)

# hitung nilai prediksi
# menghitung nilai prediksi dari user ke 0 terhadap objek ke 0

# mengambil rating user
ratings_similar_user = llor_array[index_similar,:]
# hitung rata-rata dari tiap user
mean_user_rating = ratings_similar_user.mean(axis=1)
# rata-rata dari rating ke u (rating ke 0)
mean_rating_u = llor_array[0].mean()
# ambil objek ke 0
zero_object = ratings_similar_user[:,1]

for i in range(len(zero_object)):
    predict_score = mean_rating_u + (np.sum(most_similar[i]*(zero_object[i] - mean_user_rating[i])) / np.sum(np.absolute(most_similar)))
    
# pd.DataFrame(zero_object)
# menghitung prediksi untuk user 0 terhadap object 1
# pred = mean_user_rating[0] + np.sum(most_similar * diff_ratings[:,2]) / np.sum(most_similar)
# print(pred)
# pd.DataFrame(diff_ratings)

In [91]:
print(predict_score)

0.10863925894736955


In [64]:
# rating paling similar
pd.DataFrame(ratings_similar_user).head()

,0,1,2,3,4,5,6,7,8,9,...,9714,9715,9716,9717,9718,9719,9720,9721,9722,9723
0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.5,0.0,0.0,0.0,0.0,1.8,0.0,0.0,0.0,4.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,2.7,0.0,0.0,4.0,0.0,0.0,0.0,3.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.9,0.0,0.0,0.0,0.0,3.6,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
# index yang paling similar
pd.DataFrame(most_similar)

,0
0,0.655747
1,0.662242
2,0.669745
3,0.673480
4,0.683657
5,0.696920
6,0.699041
7,0.699837
8,0.700414
9,0.705112


In [72]:
# rata-rata paling similar
pd.DataFrame(mean_user_rating).head()

,0
0,0.134472
1,0.175340
2,0.151306
3,0.101075
4,0.072624
